<div style="background-color:white;" >
<div style="clear: both; display: table;">
  <div style="float: left; width: 14%; padding: 5px; height:auto">
    <img src="img/TUBraunschweig_CO_200vH_300dpi.jpg" alt="TU_Braunschweig" style="width:100%">
  </div>
  <div style="float: left; width: 28%; padding: 5px; height:auto">
    <img src="img/TU_Clausthal_Logo.png" alt="TU_Clausthal" style="width:100%">
  </div>
  <div style="float: left; width: 25%; padding: 5px; height:auto">
    <img src="img/ostfalia.jpg" alt="Ostfalia" style="width:100%">
  </div>
  <div style="float: left; width: 21%; padding: 5px;">
    <img src="img/niedersachsen_rgb_whitebg.png" alt="Niedersachsen" style="width:100%">
  </div>
  <div style="float: left; width: 9%; padding: 5px;">
    <img src="img/internet_BMBF_gefoerdert_2017_en.jpg" alt="bmbf" style="width:100%">
  </div>
</div>
<div style="text-align:center">
<img src="img/ki4all.jpg" alt="KI4ALL-Logo" width="200"/>
</div>
</div>

# Synthetic Biomedical Data – Lesson 2a: Data Generation Basics (Part 1)

Part of the *Microcredit Artificial Data Generator* learning module.

➡️ [Back to Lesson 1: Introduction](01_intro.ipynb)
➡️ [Module Overview (README)](../README.md)

*Before continuing, please make sure you have reviewed the **prerequisites** and **learning goals** in Lesson 1.*

### Recap from Lesson 1
- Synthetic biomedical data is artificially generated but mimics statistical properties of real datasets.
- It allows us to know the *ground truth* (feature importance, class distributions).
- Useful for benchmarking, teaching, privacy-preserving research.

In Lesson 1, you learned why synthetic biomedical data is useful.
Now we create and explore our **first simple dataset**.

## Learning Goals (Lesson 2)

The goal of this lesson is to give you hands-on experience with creating small, controlled datasets that demonstrate the key concepts of synthetic biomedical data.
Later notebooks will expand this by introducing **pseudo-classes**, **random effects**, and more advanced data structures.

After completing this lesson, you will be able to:

1. **Generate** a simple synthetic biomedical dataset with controlled numbers of samples, features, and classes.
2. **Inspect** the structure of the dataset (features, labels, class balance) in a Pandas DataFrame.
3. **Visualize** data using scatterplots and histograms to explore class separability.
4. **Identify** which features appear informative and which behave like noise.
5. **Quantify** the strength of class separation using effect size measures (Cohen’s d, Cliff’s delta), and relate these values to visual impressions.
6. **Explain** why synthetic data provides a safe playground for learning and method testing before moving on to more complex cases.
---


## Lesson 2a: Data Generation Basics — First Dataset

In this notebook, we will move from theory to practice and learn how to **generate synthetic biomedical datasets** using Python.
You will see how simple tools (e.g., `scikit-learn`) can be used to create datasets that mimic the structure of real biomedical data while allowing full control over:

- **Number of samples** (patients, experiments, or observations)
- **Number of features** (biomarkers or measurements)
- **Number of classes** (e.g., healthy vs. diseased groups)
- **Feature importance** (which features carry useful information)
- **Noise and irrelevant features** (variables that add randomness or distraction)

###  Learning Goals
After completing this notebook, you will be able to:
1. Generate a synthetic dataset with `make_classification`.
2. Inspect the structure of the dataset (samples, features, labels).
3. Check the class distribution.
4. Visualize class separability with a simple scatterplot.

---

### Lesson 2a.1 — Generate a simple synthetic dataset

We will create a tiny synthetic dataset that imitates biomedical measurements:
- **Samples** ≈ patients
- **Features** ≈ biomarkers (e.g., gene expression levels)
- **Class** ≈ outcome (e.g., healthy vs. diseased)

This minimal example uses `sklearn.datasets.make_classification`, which lets us control:
- number of samples and features,
- how many features are truly informative,
- the number of classes.

> **Goal:** Get a quick, tangible dataset to explore (as a Pandas DataFrame).


In [1]:
# Lesson 2a: First Example – Generate a simple synthetic dataset

from sklearn.datasets import make_classification
import pandas as pd

# Generate a dataset with:
# - 30 samples (rows, like patients)
# - 8 features (columns, like biomarkers)
# - 2 informative features (really matter for classification)
# - 2 classes (e.g., healthy vs diseased)
X, y = make_classification(
    n_samples=30,
    n_features=8,
    n_informative=2,
    n_redundant=0,
    n_classes=2,
    class_sep=1.0,
    random_state=42
)

# Put into a DataFrame for easier handling
feature_names = [f"feature_{i+1}" for i in range(X.shape[1])]
df = pd.DataFrame(X, columns=feature_names)
df["class"] = y

# Show first rows
df.head()

,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,class
0,2.122156,0.917862,1.223877,-1.260884,-1.519370,1.032465,-0.484234,0.854515,1
1,0.570891,3.852731,1.749427,0.515048,0.954002,1.135566,0.651391,-1.718794,0
2,0.257550,0.174578,-0.667208,1.886186,-1.918771,-0.074446,-0.026514,-0.432656,0
3,0.296120,-1.463515,-1.824213,-0.392108,0.005113,0.261055,-0.234587,-2.646789,0
4,-0.990536,2.190456,-1.221743,0.586857,0.099651,-0.566298,-0.503476,0.703892,1


#### Interpretation & Takeaways
- Each **row** is a sample (patient); each **feature_*** column is a biomarker.
- The **class** column is our target (0/1). At this point, the numbers are just synthetic values—what matters is the structure.
- We specified **2 informative features**; the others may behave like noise here.

**Reflection**
- How would changing `n_informative` or `n_features` affect downstream plots and model difficulty?
- What happens if you increase `n_samples` to 1000?


## 2a.2 Explore Class Distribution

Before diving deeper into feature analysis, it is important to understand how many samples
belong to each class (e.g., **healthy** vs. **diseased**). This is called the **class distribution**.

---

### Why check class distribution?

- **Baseline understanding:**
  If one class has 80 samples and the other has 20, we already know the dataset is imbalanced.

- **Impact on machine learning models:**
  - In a **balanced dataset** (e.g., 50 healthy, 50 diseased), most models can learn both classes equally well.
  - In an **imbalanced dataset** (e.g., 95 healthy, 5 diseased), a naïve model could simply predict *all samples as healthy* and still achieve 95% accuracy — but it would be useless for detecting disease.

- **Evaluation metrics:**
  With imbalanced data, **accuracy alone is misleading**.
  We then need other metrics like **precision, recall, F1-score, ROC-AUC** to judge performance fairly.

- **Synthetic data advantage:**
  With generated data, we can **control class balance**.
  - Want to simulate a rare disease? Generate 5% diseased, 95% healthy.
  - Want to test fairness? Generate equal class sizes.

---

### What to do in this step
1. Count how many samples belong to each class (e.g., using `value_counts()` in Pandas).
2. Plot the counts (optional: bar chart or pie chart).
3. Reflect: Is the dataset balanced or skewed?

---

### Quick Takeaway
- Always start by checking class distribution.
- Class balance affects how models learn and how performance should be measured.
- Synthetic datasets let us experiment with both balanced and imbalanced cases,
  something that is often hard with real biomedical data.

In [ ]:
# Count samples per class
class_counts = df["class"].value_counts()
print("Class distribution (counts):")
print(class_counts)

# Optional: visualize as a bar chart
import seaborn as sns
import matplotlib.pyplot as plt

plt.figure(figsize=(4,4))
sns.countplot(x="class", data=df, palette="Set1")
plt.title("Class Distribution")
plt.xlabel("Class label")
plt.ylabel("Number of samples")
plt.show()

#### Interpretation
- The printed counts show how many samples belong to each class.
- The bar chart provides a quick visual overview of balance.

**Reflection**
- Is the dataset balanced or skewed?
- If one class dominates, how might this affect model training?
- In biomedical research, rare diseases often mean highly imbalanced datasets.
  How could synthetic data help simulate and study such cases?


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Scatterplot of two selected features, colored by class
plt.figure(figsize=(6,6))
sns.scatterplot(
    data=df,
    x="feature_1",
    y="feature_2",
    hue="class",
    palette="Set1",
    s=80
)
plt.title("Scatterplot of feature_1 vs feature_2 (colored by class)")
plt.xlabel("feature_1")
plt.ylabel("feature_2")
plt.legend(title="Class")
plt.show()

#### Interpretation
- Each point = one sample.
- The position shows the feature values (x = feature_1, y = feature_2).
- The color shows the class label.

If the two classes form **separate clusters**, the features are informative.
If they **overlap**, then these features alone may not be enough for good classification.

---

**Reflection**
- Try plotting different pairs of features (e.g., `feature_3` vs `feature_4`).
  - Do some feature pairs separate the classes better than others?
- What happens if you regenerate the dataset with a larger `class_sep` value?
  - Does the scatterplot show clearer separation?
- In real biomedical data, why is it rare to see perfect separation in 2D plots?


## 2a.3 Scatterplot of Two Features

So far, we have looked at class sizes, but not yet at how features
help us distinguish between classes. A **scatterplot** is a simple way
to explore **class separability** in two dimensions.

---

### What is a scatterplot?
- Each **point** in the plot = one sample (e.g., one patient).
- The **x-axis** and **y-axis** each represent a feature (e.g., biomarker_1 vs biomarker_2).
- The **color** of each point indicates the class label (e.g., healthy = red, diseased = blue).

This lets us see whether the two classes overlap or form distinct clusters.

---

### Why this matters
- If the classes are **clearly separated** in the scatterplot:
  - A simple model (like a straight line) may already do well.
- If the classes **overlap strongly**:
  - Classification will be harder.
  - We may need more complex models or additional features.
- In real biomedical data, perfect separation is rare — so synthetic data
  gives us a safe way to practice with both easy and hard cases.

---

### What to do in this step
1. Choose two feature columns (e.g., `feature_1` and `feature_2`).
2. Plot them against each other in a scatterplot.
3. Color the points by their class label.
4. Inspect whether classes cluster or overlap.

**Tip:** If the plot looks very mixed, try generating the data with a larger `class_sep` (e.g., `class_sep=2.0`) or choose a pair of more **informative** features.

---

### Quick Takeaway
A scatterplot provides an **intuitive first view** of how separable
two classes are. It builds intuition for what models will later try to do:
draw a **decision boundary** that best separates the classes.


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Scatterplot of two selected features, colored by class
plt.figure(figsize=(6,6))
sns.scatterplot(
    data=df,
    x="feature_1",
    y="feature_2",
    hue="class",
    palette="Set1",
    s=80
)
plt.title("Scatterplot of feature_1 vs feature_2 (colored by class)")
plt.xlabel("feature_1")
plt.ylabel("feature_2")
plt.legend(title="Class")
plt.show()


#### Interpretation
- Each point = one sample.
- The position shows the feature values (x = feature_1, y = feature_2).
- The color shows the class label.

If the two classes form **separate clusters**, the features are informative.
If they **overlap**, then these features alone may not be enough for good classification.

---

**Reflection**
- Try plotting different pairs of features (e.g., `feature_3` vs `feature_4`).
  - Do some feature pairs separate the classes better than others?
- What happens if you regenerate the dataset with a larger `class_sep` value?
  - Does the scatterplot show clearer separation?
- In real biomedical data, why is it rare to see perfect separation in 2D plots?


## Quick Takeaway from Lesson 2a

- We can easily **generate** a synthetic dataset with controlled numbers of samples, features, and classes.
- By checking the **class distribution**, we ensure the dataset is balanced (or intentionally imbalanced).
- A **scatterplot of two features** gives a first intuitive sense of whether classes are separable.
- In synthetic data, we can control how difficult separation is, which makes it a safe playground to explore basic ML concepts.

---

## Next Steps

In the next notebook (**Lesson 2b: Feature Distributions and Effect Size**),
we will go beyond single scatterplots and explore **all features** in more detail:

- Visualize class-conditional distributions for each feature.
- Identify which features are informative and which behave like noise.
- Quantify differences between groups using **effect size measures** (Cohen’s d, Cliff’s delta).
- Connect visual impressions to numeric measures of separation.

➡️ Continue with: **`02b_feature_distributions_effect_size.ipynb`**
